In [ ]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.metrics import mean_squared_error, r2_score

# Load data
df = pd.read_csv("Merged (Gas per jam).csv")
df.dropna(inplace=True)

# Input dan Target
input_features = ['GAS_MMBTU', 'GAS_Sm3']
target_columns = ['D101330TT (Tem.outlet chamber)', 'D102265TIC_PV (Temp. inlet chamber)', 'D102260TIC_CV (High press. Steam damper)', 'D102265TIC_CV (Low press. Steam damper)', 'D102266TIC (Main heater dehumidifier)']

# Folder model
model_dir = "saved_models_v2"

# Model type mapping
model_types = {
    "xgb": "XGBoost",
    "rf": "Random Forest",
    "dt": "Decision Tree"
}

# Menyimpan hasil evaluasi
all_results = []

for model_prefix, model_name in model_types.items():
    for target in target_columns:
        model_path = os.path.join(model_dir, f"{model_prefix}_model_{target}.pkl")
        if os.path.exists(model_path):
            model = joblib.load(model_path)

            # Split X dan y
            X = df[input_features].values
            y_true = df[target].values

            # Prediksi
            y_pred = model.predict(X)

            mse = mean_squared_error(y_true, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_true, y_pred)
            mean_actual = np.mean(y_true)
            mean_pred = np.mean(y_pred)
            accuracy_vs_mean = 100 - abs(mean_pred - mean_actual) / mean_actual * 100

            all_results.append({
                "Model": model_name,
                "Target Parameter": target,
                "MSE": round(mse, 4),
                "RMSE": round(rmse, 4),
                "R²": round(r2, 4),
                "Rata-rata Aktual": round(mean_actual, 2),
                "Rata-rata Prediksi": round(mean_pred, 2),
                "Akurasi terhadap Rata-rata (%)": round(accuracy_vs_mean, 2)
            })

# Tampilkan hasil
df_result = pd.DataFrame(all_results)
print(df_result)

# Simpan ke file
df_result.to_csv("evaluasi_semua_model.csv", index=False)
print("\n✅ Hasil evaluasi disimpan ke evaluasi_semua_model.csv")


            Model Target Parameter       MSE     RMSE      R²  \
0         XGBoost        D101330TT   69.8328   8.3566  0.9191   
1         XGBoost    D102265TIC_PV  369.1406  19.2130  0.9270   
2         XGBoost    D102260TIC_CV  130.4946  11.4234  0.8315   
3         XGBoost    D102265TIC_CV  121.3896  11.0177  0.5962   
4         XGBoost       D102266TIC    6.3095   2.5119  0.8568   
5   Random Forest        D101330TT   19.2760   4.3904  0.9777   
6   Random Forest    D102265TIC_PV   94.5192   9.7221  0.9813   
7   Random Forest    D102260TIC_CV   28.5818   5.3462  0.9631   
8   Random Forest    D102265TIC_CV   13.0803   3.6167  0.9565   
9   Random Forest       D102266TIC    0.5611   0.7490  0.9873   
10  Decision Tree        D101330TT   19.2696   4.3897  0.9777   
11  Decision Tree    D102265TIC_PV   94.5002   9.7211  0.9813   
12  Decision Tree    D102260TIC_CV   28.5735   5.3454  0.9631   
13  Decision Tree    D102265TIC_CV   13.0779   3.6163  0.9565   
14  Decision Tree       D